# Audio demos for MiniGAN web

## Initial imports

In [31]:
import torch
import torchaudio
import soundfile as sf
import os
from utilsgans import power_compress, power_uncompress
import newarchs as gans
import models
import IPython.display as ipd

## Import model and audio

In [2]:
minigan = gans.SerializableModule().create("MiniGAN generator")
minigan.load_state_dict(torch.load("C:/Users/labad/OneDrive/Documentos/muia/tfm/demo/models/checkpoints/se_gans/MiniGAN/generator.pt", map_location=torch.device('cpu')))
minigan.eval()
minigannda = gans.SerializableModule().create("MiniGAN generator")
minigannda.load_state_dict(torch.load("C:/Users/labad/OneDrive/Documentos/muia/tfm/demo/models/checkpoints/se_gans/MiniGAN-48/generator.pt", map_location=torch.device('cpu')))
minigannda.eval()
miniganft = gans.SerializableModule().create("MiniGANmod1 generator")
miniganft.load_state_dict(torch.load("C:/Users/labad/OneDrive/Documentos/muia/tfm/demo/models/checkpoints/se_gans/MiniGAN-FT/generator.pt", map_location=torch.device('cpu')))
miniganft.eval()
minigan48 = gans.SerializableModule().create("MiniGANmod2 generator")
minigan48.load_state_dict(torch.load("C:/Users/labad/OneDrive/Documentos/muia/tfm/demo/models/checkpoints/se_gans/MiniGAN-NDA/generator.pt", map_location=torch.device('cpu')))
minigan48.eval()
miniganwn = gans.SerializableModule().create("MiniGAN generator")
miniganwn.load_state_dict(torch.load("C:/Users/labad/OneDrive/Documentos/muia/tfm/demo/models/checkpoints/se_gans/MiniGAN-WN/generator.pt", map_location=torch.device('cpu')))
miniganwn.eval()
miniganedm = gans.SerializableModule().create("LitGAN generator")
miniganedm.load_state_dict(torch.load("C:/Users/labad/OneDrive/Documentos/muia/tfm/demo/models/checkpoints/se_gans/MiniGAN-EDM/generator.pt", map_location=torch.device('cpu')))
miniganedm.eval()

C:\Users\labad\AppData\Local\Temp\ipykernel_7380\3672923361.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  minigan.load_state_dict(torch.load("C:/Users/labad/OneDrive/D

LitGANGenerator(
  (dense_encoder): EncoderLit(
    (dense_conv_1): Sequential(
      (0): Conv2d(2, 4, kernel_size=(1, 1), stride=(1, 1))
      (1): InstanceNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): PReLU(num_parameters=4)
    )
    (dense_conv_2): Sequential(
      (0): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): InstanceNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): PReLU(num_parameters=8)
    )
    (dense_conv_3): Sequential(
      (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(2, 1), dilation=(2, 1))
      (1): InstanceNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): PReLU(num_parameters=16)
    )
    (dense_conv_4): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(4, 1), dilation=(4, 1))
      (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
 

In [24]:
own_noisy, sr = torchaudio.load("C:/Users/labad/OneDrive/Documentos/muia/tfm/demo/own_audio_noisy.wav")
print(sr)
test1_noisy, sr = torchaudio.load("C:/Users/labad/OneDrive/Documentos/muia/tfm/demo/test1_noisy.wav") #232_034
print(sr)
test2_noisy, sr = torchaudio.load("C:/Users/labad/OneDrive/Documentos/muia/tfm/demo/test2_noisy.wav") #232_044
print(sr)

8000
48000
48000


In [25]:
resampler_own = torchaudio.transforms.Resample(orig_freq=8000, new_freq=16000)
own_noisy = resampler_own(own_noisy)
resampler_test = torchaudio.transforms.Resample(orig_freq=48000, new_freq=16000)
test1_noisy = resampler_test(test1_noisy)
test2_noisy = resampler_test(test2_noisy)

In [27]:
ipd.Audio(own_noisy, rate=16000)

## Process audios

In [32]:
#audios_dic={"own_noisy": own_noisy, "test1_noisy": test1_noisy, "test2_noisy": test2_noisy}
audios_dic={"own_noisy": own_noisy}
result_list = []

for name, val in audios_dic.items(): 
    c = torch.sqrt(val.size(-1) / torch.sum((val**2.0), dim=-1)) #calculates the normalization factor c
    noisy_audio = torch.transpose(val, 0, 1)
    noisy_audio = torch.transpose(noisy_audio * c, 0, 1)
    noisy_spec = torch.stft(noisy_audio, 512, 100, window=torch.hamming_window(512), onesided=True, return_complex=True)
    noisy_spec = power_compress(noisy_spec)
    noisy_real = noisy_spec[:, 0, :, :].unsqueeze(1) 
    noisy_imag = noisy_spec[:, 1, :, :].unsqueeze(1) 
    noisy_phase = torch.atan2(noisy_imag.squeeze(1), noisy_real.squeeze(1))
    noisy_mag = torch.sqrt(noisy_real**2 + noisy_imag**2)

    with torch.no_grad():
        est_mag_minigan, est_phase_minigan=minigan(noisy_mag, noisy_phase)
        print("minigan", name)
        #est_mag_minigannda, est_phase_minigannda=minigannda(noisy_mag, noisy_phase)
        #print("minigannda", name)
        #est_mag_miniganft, est_phase_miniganft=miniganft(noisy_mag, noisy_phase)
        #print("miniganft", name)
        #est_mag_minigan48, est_phase_minigan48=minigan48(noisy_mag, noisy_phase)
        #print("minigan48", name)
        #est_mag_miniganwn, est_phase_miniganwn=miniganwn(noisy_mag, noisy_phase)
        #print("miniganwn", name)
        #est_mag_miniganedm, est_phase_miniganedm=miniganedm(noisy_mag, noisy_phase)
        #print("miniganedm", name)


    est_real_minigan = est_mag_minigan * torch.cos(est_phase_minigan).unsqueeze(-1).permute(0, 3, 1, 2)
    est_imag_minigan = est_mag_minigan * torch.sin(est_phase_minigan).unsqueeze(-1).permute(0, 3, 1, 2)
    est_spec_uncompress_minigan = power_uncompress(est_real_minigan, est_imag_minigan).squeeze(1)
    clean_audio_minigan =  torch.istft(est_spec_uncompress_minigan,512,100,window=torch.hamming_window(512),onesided=True)
    clean_audio_minigan_ad=clean_audio_minigan/c
    max_en = torch.max(torch.abs(clean_audio_minigan_ad))
    minigan_ad = clean_audio_minigan_ad / max_en
    result_list.append(clean_audio_minigan)
    result_list.append(clean_audio_minigan_ad)
    result_list.append(minigan_ad)

RuntimeError: Expected both inputs to be Half, Float or Double tensors but got ComplexFloat and ComplexFloat

In [5]:

names = ["eng_instrumental_LitGAN.wav", "eng_instrumental_MidGAN.wav", "eng_instrumental_MiniGAN.wav" ]
saved_dir="C:/Users/labad/OneDrive/Documentos/muia/tfm/demo"

for i, (audio, name) in enumerate(zip(result_list, names)):
  saved_path = os.path.join(saved_dir, name)
  audio_np=audio.squeeze().detach().cpu().numpy()
  sf.write(saved_path, audio_np, sr)